In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#CPCG0196

In [3]:
#Read the data

GenomicFeatures_CPCG0196 = pd.read_csv(open('data/CPCG0196/GenomicFeatures_CPCG0196.txt','r'),sep='\t')
SNVCalls_CPCG0196    = pd.read_csv(open('data/CPCG0196/SNVCalls_CPCG0196.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0196.shape
print SNVCalls_CPCG0196.shape

(262727, 15)
(262727, 153)


In [5]:
GenomicFeatures_CPCG0196.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10596,38,4,1238,54,22,15,79.5,ACA,1.860759,0.706468,intergenic,20327,0.277778
1,chr1,14464,118,42,4553,182,67,0,58.0,CTG,2.538462,0.547264,NaN,16459,0.527473
2,chr1,15484,194,23,6701,244,104,0,42.0,CCT,1.838509,0.542289,ncRNA_intronic,15439,0.446721
3,chr1,19004,66,41,2855,117,49,0,44.0,ATG,2.315436,0.577114,ncRNA_intronic,11919,0.615385
4,chr1,19279,152,20,4198,186,72,9,58.0,ACG,2.455172,0.656716,NaN,11644,0.510753


In [6]:
SNVCalls_CPCG0196.head()

,CHROM,POS,END,X2399005,X2642542,X2642549,X2673214,X2673234,X2679057,X2679060,X2788813,X2788819,X2788825,X2788828,X2788843,X2788850,X2788853,X2788856,X2791356,X2791362,
0,chr1,10595,10596,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,...
1,chr1,14463,14464,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,15483,15484,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,19003,19004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,19278,19279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [7]:
np.unique(GenomicFeatures_CPCG0196['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NTC', 'TCA', 'TCC',
       'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [8]:
CPCG0196 = pd.merge(GenomicFeatures_CPCG0196, SNVCalls_CPCG0196,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [9]:
CPCG0196.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399005,X2642542,X2642549,
0,chr1,10596,38,4,1238,54,22,15,79.5,ACA,1.860759,0.706468,intergenic,20327,0.277778,10595,10596,0,0,0,...
1,chr1,14464,118,42,4553,182,67,0,58.0,CTG,2.538462,0.547264,NaN,16459,0.527473,14463,14464,0,0,0,...
2,chr1,15484,194,23,6701,244,104,0,42.0,CCT,1.838509,0.542289,ncRNA_intronic,15439,0.446721,15483,15484,1,0,0,...
3,chr1,19004,66,41,2855,117,49,0,44.0,ATG,2.315436,0.577114,ncRNA_intronic,11919,0.615385,19003,19004,0,0,0,...
4,chr1,19279,152,20,4198,186,72,9,58.0,ACG,2.455172,0.656716,NaN,11644,0.510753,19278,19279,0,0,0,...


In [10]:
CPCG0196_X = CPCG0196.iloc[:,2:15] #Exclude CHROM, POS
CPCG0196_Y = CPCG0196.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [11]:
print CPCG0196_X.shape
print CPCG0196_Y.shape

(262727, 13)
(262727, 150)


In [12]:
CPCG0196_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,38,4,1238,54,22,15,79.5,ACA,1.860759,0.706468,intergenic,20327,0.277778
1,118,42,4553,182,67,0,58.0,CTG,2.538462,0.547264,NaN,16459,0.527473


In [13]:
CPCG0196_Y[:2]

,X2399005,X2642542,X2642549,X2673214,X2673234,X2679057,X2679060,X2788813,X2788819,X2788825,X2788828,X2788843,X2788850,X2788853,X2788856,X2791356,X2791362,X2791366,X2791371,X2791397,
0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [14]:
totalSample = CPCG0196_Y.shape[0]
CPCG0196_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0196_Y_Val.shape

262727
(262727, 1)


In [15]:
CPCG0196_Y.shape

(262727, 150)

In [16]:
CPCG0196_Y.values[0].shape

(150,)

In [17]:
for i in range(totalSample):
    CPCG0196_Y_Val[i] = np.argmax(np.bincount(CPCG0196_Y.values[i]))

In [18]:
CPCG0196_Y_Val_1 = pd.DataFrame(CPCG0196_Y_Val)
CPCG0196_Y_Val_1.columns = ['Label']

In [19]:
CPCG0196_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [20]:
CPCG0196_dataframe = pd.concat([CPCG0196_X, CPCG0196_Y_Val_1],axis=1)

In [21]:
CPCG0196_dataframe.shape

(262727, 14)

In [22]:
CPCG0196_dataframe.to_csv('CPCG0196.csv')

In [23]:
#Now we have variable importance ...lets do analysis

In [24]:
%load_ext rpy2.ipython

In [26]:
%%R
# Read in the data
CPCG0196Frame = read.csv("CPCG0196.csv")
str(CPCG0196Frame)

'data.frame':	262727 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  38 118 194 66 152 96 95 92 119 71 ...
 $ NonRefAllele   : int  4 42 23 41 20 67 22 18 31 1 ...
 $ BaseQual       : int  1238 4553 6701 2855 4198 4669 3650 3245 4428 1991 ...
 $ TumourCoverage : int  54 182 244 117 186 188 140 131 183 88 ...
 $ NormalCoverage : int  22 67 104 49 72 79 70 62 70 39 ...
 $ MapQual        : num  15 0 0 0 9 0 0 0 0 0 ...
 $ ReadPosition   : num  79.5 58 42 44 58 38 54.5 62 54 45 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 1 15 12 7 3 29 4 2 17 12 ...
 $ HomopolymerRate: num  1.86 2.54 1.84 2.32 2.46 ...
 $ GCcontent      : num  0.706 0.547 0.542 0.577 0.657 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 5 1 8 8 1 8 1 8 1 8 ...
 $ DistanceSNP    : int  20327 16459 15439 11919 11644 11065 2580 2569 2412 1947 ...
 $ StrandBias     : num  0.278 0.527 0.447 0.615 0.511 ...
 $ Label          : num  0 0 0 0 0 0

In [27]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
sum(is.na(CPCG0196Frame$ReadPosition)) 

[1] 48


In [28]:
%%R
CPCG0196Frame$ReadPosition[is.na(CPCG0196Frame$ReadPosition)] = mean(CPCG0196Frame$ReadPosition, na.rm=TRUE)
CPCG0196Frame$StrandBias[is.na(CPCG0196Frame$StrandBias)] = mean(CPCG0196Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0196Frame$ReadPosition)) )
print (sum(is.na(CPCG0196Frame$StrandBias)) )

[1] 0
[1] 0


In [29]:
%%R
sum(is.na(CPCG0196Frame))  #No NA value now

[1] 0


In [30]:
%%R
write.csv(CPCG0196Frame, file = "CPCG0196_imputed.csv")  #Cleaned CSV without NA

In [31]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         42 minutes 40 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_szm488 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.58 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [32]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0196_imputed.csv"
CPCG0196.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0196.hex") 
train                 = as.data.frame(CPCG0196.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [33]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [34]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.263849
2 NormalCoverage            0.183256          0.183256   0.048352
3       BaseQual            0.166853          0.166853   0.044024
4 TumourCoverage            0.151505          0.151505   0.039975
5      RefAllele            0.147368          0.147368   0.038883

---
                         variable relative_importance scaled_importance
56              Trinucleotide.ACT            0.015940          0.015940
57 GenomicLocation.ncRNA_intronic            0.015032          0.015032
58     GenomicLocation.intergenic            0.010248          0.010248
59       GenomicLocation.intronic            0.010142          0.010142
60                      GCcontent            0.007797          0.007797
61      Trinucleotide.missing(NA)            0.000000          0.000000
   percentage
56   0.004206
57   0.003966
58   0.002704
59   0.002676
60

In [35]:
%%R
write.table(var_importance, file="CPCG0196_variable_importance.csv")

In [36]:
var_importance = pd.read_csv(open('CPCG0196_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [37]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.263849
2,NormalCoverage,0.183256,0.183256,0.048352
3,BaseQual,0.166853,0.166853,0.044024
4,TumourCoverage,0.151505,0.151505,0.039975
5,RefAllele,0.147368,0.147368,0.038883


In [38]:
GenomicFeatures_CPCG0196[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10596,38,4,1238,54,22,15,79.5,ACA,1.860759,0.706468,intergenic,20327,0.277778
1,chr1,14464,118,42,4553,182,67,0,58.0,CTG,2.538462,0.547264,NaN,16459,0.527473
2,chr1,15484,194,23,6701,244,104,0,42.0,CCT,1.838509,0.542289,ncRNA_intronic,15439,0.446721
3,chr1,19004,66,41,2855,117,49,0,44.0,ATG,2.315436,0.577114,ncRNA_intronic,11919,0.615385
4,chr1,19279,152,20,4198,186,72,9,58.0,ACG,2.455172,0.656716,NaN,11644,0.510753


In [39]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.166853487491608,
 'DistanceSNP': 0.067449428141117096,
 'GCcontent': 0.0077970633283257502,
 'GenomicLocation.UTR3': 0.0248018726706505,
 'GenomicLocation.UTR5': 0.062692426145076793,
 'GenomicLocation.downstream': 0.081780396401882199,
 'GenomicLocation.exonic': 0.022973190993070599,
 'GenomicLocation.exonic;splicing': 0.055495861917734098,
 'GenomicLocation.intergenic': 0.010247824713587801,
 'GenomicLocation.intronic': 0.010142472572624701,
 'GenomicLocation.missing(NA)': 0.0195613969117403,
 'GenomicLocation.ncRNA_UTR3': 0.054753314703702899,
 'GenomicLocation.ncRNA_UTR5': 0.050459135323762901,
 'GenomicLocation.ncRNA_exonic': 0.044068817049264901,
 'GenomicLocation.ncRNA_intronic': 0.0150324692949653,
 'GenomicLocation.ncRNA_splicing': 0.0873107239603996,
 'GenomicLocation.splicing': 0.046353604644537,
 'GenomicLocation.upstream': 0.0275620929896832,
 'GenomicLocation.upstream;downstream': 0.050399985164403902,
 'HomopolymerRate': 0.023289060220122299,
 'MapQual': 0

In [41]:
noOfSample = GenomicFeatures_CPCG0196.shape[0]

CPCG0196_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0196.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0196.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0196.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0196.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0196.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0196.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0196.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0196.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0196.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0196.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0196.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0196.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0196.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0196.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0196.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0196.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0196_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [42]:
#Sorting dictionary
CPCG0196_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0196_Dict.items()],reverse=True)]

In [43]:
CPCG0196_Dict_Frame = pd.DataFrame(CPCG0196_Dict1, columns=['POS', 'Value'])

In [44]:
CPCG0196_Dict_Frame.to_csv("CPCG0196_Dict_Frame.csv")

In [22]:
CPCG0196_Dict_Frame = pd.read_csv("CPCG0196_Dict_Frame.csv")

In [23]:
CPCG0196_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,181078,170080.740476
1,1,303265,161703.666077
2,2,423119,153768.321146
3,3,576476,143364.729163
4,4,579326,143122.632771
5,5,797966,128569.166857
6,6,878548,122968.125671
7,7,919927,120243.583833
8,8,1369555,89897.923831
9,9,30032083,87192.700699


In [24]:
GenomicFeatures_CPCG0196_1 = pd.concat([CPCG0196.iloc[:,0:16], CPCG0196_Y_Val_1],axis=1)

In [26]:
rows = CPCG0196_Dict_Frame['POS'][:10].values
print rows
GenomicFeatures_CPCG0196_Top10 = GenomicFeatures_CPCG0196_1.loc[GenomicFeatures_CPCG0196_1['POS_x'].isin(rows)]

[  181078   303265   423119   576476   579326   797966   878548   919927
  1369555 30032083]


In [28]:
GenomicFeatures_CPCG0196_Top10[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
190668,chr6,30032083,10,0,279,11,6,0,27.0,ATT,6.297297,0.263682,intronic,1291941,0.636364,30032082,0
243441,chrX,181078,38,1,1186,47,24,0,49.0,ACC,2.876812,0.542289,intergenic,2518372,0.319149,181077,0
243445,chrX,303265,14,2,423,23,7,0,74.0,ACG,2.798507,0.716418,intronic,2396185,0.478261,303264,0
243453,chrX,423119,42,0,1276,47,15,0,54.0,ACA,1.370166,0.492537,intergenic,2276331,0.425532,423118,0
243461,chrX,576476,32,0,913,48,25,0,60.5,TTG,3.429630,0.507463,intergenic,2122974,0.666667,576475,0
243462,chrX,579326,14,9,610,28,14,0,45.5,TCC,2.000000,0.407960,intergenic,2120124,0.642857,579325,0
243476,chrX,797966,52,3,1726,66,27,0,38.0,TCT,2.308725,0.363184,intergenic,1901484,0.545455,797965,0
243482,chrX,878548,26,1,804,31,22,0,40.0,TCA,1.796296,0.457711,intergenic,1820902,0.451613,878547,0
243489,chrX,919927,37,6,1130,48,32,0,66.5,CTA,2.917293,0.432836,intergenic,1779523,0.437500,919926,0
243506,chrX,1369555,34,0,1068,43,27,0,66.0,TCA,1.407821,0.293532,intergenic,1329895,0.534884,1369554,0
